카카오톡 텍스트를 받아서 나와 상대방으로 챗봇을 만들어보았어요.
아직 초짜라 이것저것 그냥 보고 배낀것들이에요 양해 부탁드려요.
중간중간에 주석 되어있는 부분은 자기 컴퓨터에 해당되는 것들로 바꿔주세요.
1대1 채팅만 만들어보았으니 참고해주시고 카카오톡 1대1 방에서 채팅 내보내기한 텍스트 파일로 만들어본것입니다.

전처리 과정

In [ ]:
import re
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

def katalk_msg_parse(file_path):
    my_katalk_data = list()
    katalk_msg_pattern = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2},.*:"
    date_info = "[0-9]{4}년 [0-9]{1,2}월 [0-9]{1,2}일 \S요일"
    in_out_info = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2}:.*"


    for line in open(file_path, encoding="utf-8"):
        if re.match(date_info, line) or re.match(in_out_info, line):
            continue
        elif line == '\n':
            continue
        elif re.match(katalk_msg_pattern, line):
            line = line.split(",")
            date_time = line[0]
            user_text = line[1].split(" : ", maxsplit=1)
            user_name = user_text[0].strip()
            text = user_text[1].strip()
            my_katalk_data.append({'date_time': date_time,
                                   'user_name': user_name,
                                   'text': text
                                   })

        else:
            if len(my_katalk_data) > 0:
                my_katalk_data[-1]['text'] += "\n" + line.strip()


    my_katalk_df = pd.DataFrame(my_katalk_data)
    return my_katalk_df


f_path = 'text_data.txt'  #가져올 카카오톡 텍스트 파일을 적어주세요
df = katalk_msg_parse(os.path.join(f_path))


save_path = 'csv_data.csv' #전처리를 완료할 csv파일이름을 적어주세요




my_user_name = ""  #  "" 사이에 자신의 카카오톡 이름을 적어주세요


df['user_name'] = df['user_name'].apply(lambda x: 'me' if x == my_user_name else ('you'))


df['group'] = (df['user_name'] != df['user_name'].shift()).cumsum()
df_grouped = df.groupby('group').agg({'user_name': 'first', 'text': lambda x: " ".join(x)}).reset_index()



mask_me_rows = df_grouped['user_name'] == "me"
mask_next_row_after_me_rows = mask_me_rows.shift(-1)
mask_next_row_after_me_rows.fillna(False, inplace=True)
responses = df_grouped.loc[mask_next_row_after_me_rows, "text"].values.tolist()
responses.append('')

df_grouped.loc[mask_me_rows, "response"] = responses


response_indices_to_drop = mask_next_row_after_me_rows[mask_next_row_after_me_rows].index.values.tolist()
if len(df_grouped) in response_indices_to_drop:
    response_indices_to_drop.remove(len(df_grouped))




delete_columns = ['user_name', 'group']
delete_rows = response_indices_to_drop

df_grouped.drop(columns=delete_columns, inplace=True)
df_grouped.drop(delete_rows, inplace=True)

df_final = df_grouped

df_final.to_csv(save_path, index=False)


df_final


In [ ]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

df = pd.read_csv(r"test_data.csv")

df.head()

In [ ]:
print(df.loc[0, 'text'])
model.encode(df.loc[0, 'text'])

In [ ]:
df['embedding'] = pd.Series([[]] * len(df))

df['embedding'] = df['text'].map(lambda x: list(model.encode(x)))

df.head()

In [ ]:
df.to_csv('test_reult_(1).csv', index = False)

In [ ]:
while True:
    enter = input("뭐라고? :")

    embedding = model.encode(enter)

    df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

    answer = df.loc[df['distance'].idxmax()]

    print('질문 : ', enter)
    print('상대방 답변 : ', answer['response'])
    print('유사도 : ', answer['distance'])